<a href="https://colab.research.google.com/github/danielhcg/AntiSmishGPT/blob/main/llama_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install llamaapi

In [2]:
import json
import pandas as pd
from llamaapi import LlamaAPI
from google.colab import userdata
import time
import requests

In [3]:
# Initialize the SDK
llama = LlamaAPI(userdata.get('LLAMA-API-KEY'))

In [4]:
def detect(sender, message_body, index, total):
    print(f"Evaluating message {index} of {total}...")  #progress indicator

    #converts each input into json compliant strings
    sender = json.dumps(sender)
    message_body = json.dumps(message_body)

    #build api request
    api_request_json = {
        "messages": [
            {"role": "user", "content": f"Is this message real or fake: {message_body} [Sender: {sender}]"}
        ],

        #defining server side scripting function
        "functions": [
            {
                "name": "detect_fraudulent_sms",
                "description": "Analyze if an SMS message is legitimate or a scam",
                "parameters": {
                    "type": "object",
                    "properties": { #function arguments
                        "sender": {"type": "string"},
                        "body": {"type": "string"}
                    },
                    "required": ["sender", "body"]
                }
            }
        ],
        "stream": False, #server sends entire response, instead of piece by piece
        "function_call": "detect_fraudulent_sms", #function call
        "function_parameters": {"sender": sender, "body": message_body} #specifyin parameters to send with function call
    }

    #infinite loop that breaks until successful processing of input
    while True:
        try:
            response = llama.run(api_request_json) #execute api request
            if response.status_code == 200: #checks if api request was success (200=successful http response code)
                results = response.json() #parse json data, create dictionary to work with
                message_content = results['choices'][0]['message'].get('content') #access response
                if message_content is not None:
                    return "Real" if "real" in message_content.lower() else "Fake" #handles real/fake determinations
                else:
                    print(f"Message {index}: Content was None, retrying...")
            else:
                print(f"HTTP Status Code: {response.status_code} for message {index}. Response Content: {response.text}, retrying...")
        #handling different types of errors, displaying message and retry
        except json.JSONDecodeError as e:
            print(f"JSON Decode Error for message {index}: {str(e)}, retrying...")
        except Exception as e:
            print(f"Unhandled exception for message {index}: {str(e)}, retrying...")

In [5]:
#reads input excel file, creates a copy and updates copy
def updateExcelFile(inputFilePath,outputFilePath):
    # Read the input Excel file into pandas dataframe
    df = pd.read_excel(inputFilePath, header=None, names=['Sender', 'Body'])
    total = len(df)
    results = [] #stores determinations

    # Iterate over each row in the dataframe
    for i, row in df.iterrows():
        result = detect(row['Sender'], row['Body'], i+1, total)
        results.append(result)
        print(f"Processed message {i+1} of {total}. Result: {result}")

    # Add the results to the dataframe
    df['Result'] = results
    df.to_excel(outputFilePath, index=False, header=False)  #write to new excel file
    print("All messages have been evaluated and the results are saved to the new Excel file.")


In [7]:
# Example usage: specify the path to your Excel file to read from and the new file to write to
updateExcelFile("smishTankSet.xlsx", "updated_smishTankSet.xlsx")

Evaluating message 1 of 1089...
Processed message 1 of 1089. Result: Fake
Evaluating message 2 of 1089...
Processed message 2 of 1089. Result: Fake
Evaluating message 3 of 1089...
Processed message 3 of 1089. Result: Fake
Evaluating message 4 of 1089...
Processed message 4 of 1089. Result: Fake
Evaluating message 5 of 1089...
Processed message 5 of 1089. Result: Fake
Evaluating message 6 of 1089...
Message 6: Content was None, retrying...
Processed message 6 of 1089. Result: Fake
Evaluating message 7 of 1089...
Processed message 7 of 1089. Result: Fake
Evaluating message 8 of 1089...
Processed message 8 of 1089. Result: Fake
Evaluating message 9 of 1089...
Processed message 9 of 1089. Result: Fake
Evaluating message 10 of 1089...
Processed message 10 of 1089. Result: Fake
Evaluating message 11 of 1089...
Processed message 11 of 1089. Result: Fake
Evaluating message 12 of 1089...
Message 12: Content was None, retrying...
Processed message 12 of 1089. Result: Fake
Evaluating message 13 o